In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
from google.colab.patches import cv2_imshow

In [ ]:
mainDIR = os.listdir('/content/drive/MyDrive/chest_xray 3c [AUG] (splited .85 .075 .075)')
print(mainDIR)

['train', 'val', 'test']


In [ ]:
train_folder= '/content/drive/MyDrive/chest_xray 3c [AUG] (splited .85 .075 .075)/train'
val_folder = '/content/drive/MyDrive/chest_xray 3c [AUG] (splited .85 .075 .075)/val'
test_folder = '/content/drive/MyDrive/chest_xray 3c [AUG] (splited .85 .075 .075)/test'

In [ ]:
# train
train_n = train_folder+'/Normal/'
train_b = train_folder+'/Bacterial/'
train_v = train_folder+'/Viral/'

# **Adding Noise**

In [ ]:
#get the minimum size
h_min = 100000
w_min = 100000
for train_folder in [train_n,train_b,train_v]:
  for img in os.listdir(train_folder):
    x,y,z = (cv2.imread(train_folder+img)).shape
    if x < h_min :
      h_min = x
    if y < w_min :
      w_min = y
print("H-min "+str(h_min))
print("W-min "+str(w_min))

H-min 127
W-min 224


In [ ]:
#Gaussian noise
def add_gaussian_noise(image, mean):
    """
    Add Gaussian noise to an RGB image
    """
    x,y,z = image.shape
    if x < h_min :
      std_dev = 10
    else :
      std_dev = 50
    noisy_image = image.astype('float64') + np.random.normal(mean, std_dev, image.shape)
    noisy_image = np.uint8(np.clip(noisy_image, 0, 255))
    noisy_image  = cv2.cvtColor(noisy_image, cv2.COLOR_BGR2GRAY)
    noisy_image  = cv2.cvtColor(noisy_image, cv2.COLOR_GRAY2BGR)
    return noisy_image

In [ ]:
#Poisson noise
from scipy import ndimage
def add_poisson_noise(image):
    """
    Add Poisson noise to an RGB image
    """
    noisy_image = np.zeros_like(image)
    for i in range(3):
        noisy_image[:,:,i] = np.random.poisson(image[:,:,i].astype('float64'))
    noisy_image = np.uint8(np.clip(noisy_image, 0, 255))
    noisy_image  = cv2.cvtColor(noisy_image, cv2.COLOR_BGR2GRAY)
    noisy_image  = cv2.cvtColor(noisy_image, cv2.COLOR_GRAY2BGR)
    return noisy_image

In [ ]:
from prettytable import PrettyTable
# statestics after spliting the dataset and adding noise
# stats train
num_train_normal = len(os.listdir('/content/drive/MyDrive/chest_xray 3c [AUG] (splited .85 .075 .075)/train/Normal'))
num_train_bacterial = len(os.listdir('/content/drive/MyDrive/chest_xray 3c [AUG] (splited .85 .075 .075)/train/Bacterial'))
num_train_viral = len(os.listdir('/content/drive/MyDrive/chest_xray 3c [AUG] (splited .85 .075 .075)/train/Viral'))
TOT_train = num_train_normal + num_train_bacterial + num_train_viral
# stats test
num_test_normal = len(os.listdir('/content/drive/MyDrive/chest_xray 3c [AUG] (splited .85 .075 .075)/test/Normal'))
num_test_bacterial = len(os.listdir('/content/drive/MyDrive/chest_xray 3c [AUG] (splited .85 .075 .075)/test/Bacterial'))
num_test_viral = len(os.listdir('/content/drive/MyDrive/chest_xray 3c [AUG] (splited .85 .075 .075)/test/Viral'))
TOT_test = num_test_viral + num_test_bacterial + num_test_normal
# stats valdidation
num_valid_normal = len(os.listdir('/content/drive/MyDrive/chest_xray 3c [AUG] (splited .85 .075 .075)/val/Normal'))
num_valid_bacterial = len(os.listdir('/content/drive/MyDrive/chest_xray 3c [AUG] (splited .85 .075 .075)/val/Bacterial'))
num_valid_viral = len(os.listdir('/content/drive/MyDrive/chest_xray 3c [AUG] (splited .85 .075 .075)/val/Viral'))
TOT_valid = num_valid_normal + num_valid_bacterial + num_valid_viral

TOT_normal =  num_valid_normal + num_test_normal + num_train_normal
TOT_viral = num_valid_viral + num_test_viral + num_train_viral
TOT_bacterial = num_train_bacterial + num_test_bacterial + num_valid_bacterial
TOT_all = TOT_normal + TOT_viral +TOT_bacterial

#table stat general
t = PrettyTable(['Class','Train','Validation', 'Test','Total'])
t.add_row(['Normal',num_train_normal,num_valid_normal,num_test_normal,TOT_normal])
t.add_row(['viral',num_train_viral,num_valid_viral,num_test_viral,TOT_viral])
t.add_row(['Bacterial',num_train_bacterial,num_valid_bacterial,num_test_bacterial,TOT_bacterial])
t.add_row(['Total',TOT_train,TOT_valid,TOT_test,TOT_all])
print(t)

+-----------+-------+------------+------+-------+
|   Class   | Train | Validation | Test | Total |
+-----------+-------+------------+------+-------+
|   Normal  |  1345 |    118     | 120  |  1583 |
|   viral   |  2538 |    111     | 113  |  2762 |
| Bacterial |  2363 |    208     | 209  |  2780 |
|   Total   |  6246 |    437     | 442  |  7125 |
+-----------+-------+------------+------+-------+


In [ ]:
import random
# we work on augmented data
percent = 0.2
precent_gauss = 0.6
nbr_total_normal_train =len(os.listdir('/content/drive/MyDrive/chest_xray 3c [AUG] (splited .85 .075 .075)/train/Normal'))
nbr_total_bacterial_train =len(os.listdir('/content/drive/MyDrive/chest_xray 3c [AUG] (splited .85 .075 .075)/train/Bacterial'))
nbr_total_viral_train =len(os.listdir('/content/drive/MyDrive/chest_xray 3c [AUG] (splited .85 .075 .075)/train/Viral'))

list_chosen_normal = random.sample(os.listdir('/content/drive/MyDrive/chest_xray 3c [AUG] (splited .85 .075 .075)/train/Normal'), k=int(nbr_total_normal_train*percent))
list_chosen_bacterial = random.sample(os.listdir('/content/drive/MyDrive/chest_xray 3c [AUG] (splited .85 .075 .075)/train/Bacterial'), k=int(nbr_total_bacterial_train*percent))
list_chosen_viral = random.sample(os.listdir('/content/drive/MyDrive/chest_xray 3c [AUG] (splited .85 .075 .075)/train/Viral'), k=int(nbr_total_viral_train*percent))
print("Chosen Normal radios :"+str(len(list_chosen_normal)))
print("Chosen Bacterial radios :"+str(len(list_chosen_bacterial)))
print("Chosen Viral radios :"+str(len(list_chosen_viral)))

Chosen Normal radios :269
Chosen Bacterial radios :472
Chosen Viral radios :507


In [ ]:
print(len(np.unique(list_chosen_normal)))

269


In [ ]:
print(len(np.unique(list_chosen_bacterial)))

472


In [ ]:
print(len(np.unique(list_chosen_viral)))

507


In [ ]:
cpt_gn = 0
for i in range(int(nbr_total_normal_train*percent)):
    #Normal
    img_n = cv2.imread('/content/drive/MyDrive/chest_xray 3c [AUG] (splited .85 .075 .075)/train/Normal/'+list_chosen_normal[i])
    if cpt_gn <= int(nbr_total_normal_train * percent * precent_gauss) :
      img_res =  add_gaussian_noise(img_n, mean=0)
      value = cv2.imwrite('/content/drive/MyDrive/noise 3c chest_ray .85 .075 .075/Normal/gaus_'+list_chosen_normal[i], img_res)
      cpt_gn =cpt_gn +1
    else :
      img_res =  add_poisson_noise(img_n)
      value = cv2.imwrite('/content/drive/MyDrive/noise 3c chest_ray .85 .075 .075/Normal/pois_'+list_chosen_normal[i], img_res)

In [ ]:
cpt_gb = 0
for i in range(int(nbr_total_bacterial_train*percent)):
    img_b = cv2.imread('/content/drive/MyDrive/chest_xray 3c [AUG] (splited .85 .075 .075)/train/Bacterial/'+list_chosen_bacterial[i])
    if cpt_gb <= int(nbr_total_bacterial_train * percent * precent_gauss) :
      img_res =  add_gaussian_noise(img_b, mean=0)
      val = cv2.imwrite('/content/drive/MyDrive/noise 3c chest_ray .85 .075 .075/Bacterial/gaus_'+list_chosen_bacterial[i], img_res)
      if not val:
            print(cpt_gb)
            print('WARNING')
            break;
      cpt_gb =cpt_gb+1
    else :
      img_res =  add_poisson_noise(img_b)
      val = cv2.imwrite('/content/drive/MyDrive/noise 3c chest_ray .85 .075 .075/Bacterial/pois_'+list_chosen_bacterial[i], img_res)
      if not val:
            print('WARNING')
            break;

In [ ]:
cpt_gv = 0
for i in range(int(nbr_total_viral_train*percent)):
    img_p = cv2.imread('/content/drive/MyDrive/chest_xray 3c [AUG] (splited .85 .075 .075)/train/Viral/'+list_chosen_viral[i])
    if cpt_gv <= int(nbr_total_viral_train * percent * precent_gauss) :
      img_res =  add_gaussian_noise(img_p, mean=0)
      cv2.imwrite('/content/drive/MyDrive/noise 3c chest_ray .85 .075 .075/Viral/gaus_'+list_chosen_viral[i], img_res)
      cpt_gv =cpt_gv+1
    else :
      img_res =  add_poisson_noise(img_p)
      cv2.imwrite('/content/drive/MyDrive/noise 3c chest_ray .85 .075 .075/Viral/pois_'+list_chosen_viral[i], img_res)

In [ ]:
print(len(os.listdir('/content/drive/MyDrive/noise 3c chest_ray .85 .075 .075/Normal')))

269


In [ ]:
print(len(os.listdir('/content/drive/MyDrive/noise 3c chest_ray .85 .075 .075/Bacterial')))

472


In [ ]:
print(len(np.unique(os.listdir('/content/drive/MyDrive/noise 3c chest_ray .85 .075 .075/Bacterial'))))

472


In [ ]:
print(len(os.listdir('/content/drive/MyDrive/noise 3c chest_ray .85 .075 .075/Viral')))